# Creating GraphFrame

In [1]:
import pyspark
from delta import configure_spark_with_delta_pip

import pyspark.sql.functions as F
from pyspark.sql.functions import col, array, explode, sum as Fsum
from pyspark.sql.functions import array_intersect, size
from pyspark.sql.window import Window


# Prepare the Spark builder
builder = pyspark.sql.SparkSession.builder.appName("Graphs_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder,extra_packages=["graphframes:graphframes:0.8.4-spark3.5-s_2.12"]).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)

spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)
import graphframes as gf



In [2]:
flights_df= spark.read.csv("input/2009.csv", header=True, inferSchema=True)
display(flights_df.limit(10))

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL


In [3]:

origin_airports = flights_df.select(F.col("ORIGIN").alias("id"))
dest_airports = flights_df.select(F.col("DEST").alias("id"))

vertices = origin_airports.union(dest_airports).distinct() # find set of all airports from either dest or origin column
#vertices.show()             

In [4]:
edges = flights_df.select(
    F.col("ORIGIN").alias("src"),
    F.col("DEST").alias("dst")
).distinct() # without distinct, triangles do not make much sense?

full_edges = flights_df.select(
    F.col("ORIGIN").alias("src"),
    F.col("DEST").alias("dst") # but depending on research question, both can be useful for other metrics 
)
#edges.show()


In [5]:
flights_graph = gf.GraphFrame(vertices, edges)
full_flights_graph = gf.GraphFrame(vertices, full_edges)
vertices.cache()
edges.cache()
full_edges.cache()
display(flights_graph)

GraphFrame(v:[id: string], e:[src: string, dst: string])

# Query 1

In [6]:
#simple degree calculations (structure of reachability):

in_degree = edges.groupBy("dst").count() \
    .withColumnRenamed("dst", "id") \
    .withColumnRenamed("count", "inDegree")

out_degree = edges.groupBy("src").count() \
    .withColumnRenamed("src", "id") \
    .withColumnRenamed("count", "outDegree")

total_degrees = in_degree.join(out_degree, "id", "outer") \
    .withColumn("totalDegree", F.col("inDegree") + F.col("outDegree"))

#degrees calculation(intesity)

full_in_degree = full_edges.groupBy("dst").count() \
    .withColumnRenamed("dst", "id") \
    .withColumnRenamed("count", "inDegree")

full_out_degree = full_edges.groupBy("src").count() \
    .withColumnRenamed("src", "id") \
    .withColumnRenamed("count", "outDegree")

full_total_degrees = full_in_degree.join(full_out_degree, "id", "outer") \
    .withColumn("totalDegree", F.col("inDegree") + F.col("outDegree"))

In [7]:
#triangles # only without duplicates 
undirected_edges = edges.union(edges.selectExpr("dst as src", "src as dst")).distinct() # default counter assumes graph is undirected

neighbours = undirected_edges.groupBy("src").agg(F.collect_set("dst").alias("neighbors")) # find a set of all options to go from airport
#neighbours.show()
neighbours.cache()

joined = undirected_edges \
    .join(neighbours, on="src") \
    .join(
        neighbours.withColumnRenamed("src", "dst").withColumnRenamed("neighbors", "dst_neighbors"),
        on="dst"
    )
#joined.select("src", "dst", "neighbors", "dst_neighbors").show()

with_common = joined.withColumn(
    "common_neighbors", size(array_intersect("neighbors", "dst_neighbors"))
)
#with_common.select("src", "dst", "common_neighbors").show()
triangle_counts = with_common.groupBy("src").agg(
    (F.sum("common_neighbors") / 2).cast("int").alias("manual_count")
)
triangle_counts = triangle_counts.withColumnRenamed("src", "id")
triangle_counts = triangle_counts.withColumnRenamed("manual_count", "count") # to match in assert

#triangle_counts.show()


In [8]:
# USED FOR VALIDATION ONLY: build in functions to verify results:
to_test = True # can be changed to false to skip testing
if(to_test):
    in_degrees = flights_graph.inDegrees
    out_degrees = flights_graph.outDegrees
    assert(sorted(in_degree.collect()) == sorted(in_degrees.collect()))
    assert(sorted(out_degree.collect()) == sorted(out_degrees.collect()))
    triangles = flights_graph.triangleCount().select("id","count") # reorder columns to pass assert
    assert(sorted(triangle_counts.collect()) == sorted(triangles.collect()))

    full_in_degrees = full_flights_graph.inDegrees
    full_out_degrees = full_flights_graph.outDegrees
    assert(sorted(full_in_degree.collect()) == sorted(full_in_degrees.collect()))
    assert(sorted(full_out_degree.collect()) == sorted(full_out_degrees.collect()))


In [11]:
answer = total_degrees.join(triangle_counts, "id", "left")
final_answer= answer.join(full_total_degrees,"id", "left")
final_answer.show() #showing degrees for both single and multi edged graphs

+---+--------+---------+-----------+-----+--------+---------+-----------+
| id|inDegree|outDegree|totalDegree|count|inDegree|outDegree|totalDegree|
+---+--------+---------+-----------+-----+--------+---------+-----------+
|ABQ|      32|       31|         63|  311|   35577|    35582|      71159|
|ACK|       2|        1|          3|    1|     343|      342|        685|
|ALO|       1|        1|          2|    0|     331|      330|        661|
|ANC|      28|       27|         55|  122|   17788|    17791|      35579|
|AEX|       4|        4|          8|    6|    2948|     2947|       5895|
|AKN|       1|        1|          2|    0|      77|       77|        154|
|AUS|      36|       37|         73|  416|   41846|    41843|      83689|
|ABY|       1|        1|          2|    0|     997|      995|       1992|
|ACV|       5|        4|          9|    6|    3364|     3370|       6734|
|ADK|       1|        1|          2|    0|     103|      103|        206|
|ABE|       8|        7|         15|  

# Query 2

In [12]:
count = triangle_counts.agg((F.sum("count") /3).alias("total num. of triangles")) # sum all and divide by 3 as each triangle is made of 3 airports counting it
count.show()

+-----------------------+
|total num. of triangles|
+-----------------------+
|                16015.0|
+-----------------------+



# Query 3

The chosen centrality measure is Eigenvector Centrality. It measures a node’s importance based on how well-connected its neighbors are. Unlike PageRank, it does not use a damping factor and treats all influence as coming directly from connected nodes without accounting for jumps or teleportation from further nodes. In the context of flights, it allows to measure centrality based on direct flights from popular airports.

In [13]:
# Initialize all nodes with score 1.0
v = vertices.withColumn("score", F.lit(1.0))

num_of_iterations = 10

for i in range(num_of_iterations):
    # Join edges with vertex scores to propagate influence
    joined = edges.join(v, edges.src == v.id) \
                  .select("dst", "score")

    # Sum incoming scores for each destination node
    updated_scores = joined.groupBy("dst") \
                           .agg(F.sum("score").alias("new_score"))

    # Normalize scores (done to keep the scores within a consistent range, ensuring that they remain comparable)
    norm = updated_scores.agg(F.sum("new_score").alias("norm_val")).collect()[0]["norm_val"]
    updated_scores = updated_scores.withColumn("score", F.col("new_score") / F.lit(norm)) \
                                   .select(F.col("dst").alias("id"), "score")

    # Update the vertex scores with the new scores for the next iteration, filling any missing values with 0.0
    v = vertices.join(updated_scores, "id", "left_outer") \
                .fillna(0.0, subset=["score"])

eigenvector_centrality = v.orderBy("score", ascending=False)
eigenvector_centrality.show()


+---+--------------------+
| id|               score|
+---+--------------------+
|ATL| 0.01884168732768111|
|ORD|0.018110084675545444|
|DFW|0.017016256953545333|
|DTW|0.016770254346926333|
|DEN|0.016185588480572572|
|MSP| 0.01616749521473975|
|IAH|0.015682789267655338|
|CVG| 0.01512316035975816|
|LAS|0.014596222840050355|
|EWR|0.014355588217040303|
|PHX|0.014328265155340731|
|MEM|0.014090583475358685|
|MCO|0.013613478113955267|
|CLT|0.013598602138313867|
|LAX|0.013278321168654492|
|SLC|0.013227605218655115|
|BWI|0.013205945160437514|
|IAD|0.013022968172385114|
|CLE|0.012996524760204803|
|JFK|0.012822879580061723|
+---+--------------------+
only showing top 20 rows



In [16]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(edges.collect())
centrality_nx = nx.eigenvector_centrality_numpy(G)
for row in eigenvector_centrality.collect():
    print(row["id"], row["score"], centrality_nx[row["id"]])

ATL 0.01884168732768111 0.20221386660120053
ORD 0.018110084675545444 0.1943599535105242
DFW 0.017016256953545333 0.18261773974759984
DTW 0.016770254346926333 0.1799782688459056
DEN 0.016185588480572572 0.1736968173474619
MSP 0.01616749521473975 0.17350597052269107
IAH 0.015682789267655338 0.1683027746799775
CVG 0.01512316035975816 0.16229671131312087
LAS 0.014596222840050355 0.15663399979205928
EWR 0.014355588217040303 0.154058554731138
PHX 0.014328265155340731 0.15375785648848134
MEM 0.014090583475358685 0.15121358619512948
MCO 0.013613478113955267 0.14608799360606628
CLT 0.013598602138313867 0.14593040442520808
LAX 0.013278321168654492 0.14248838885915166
SLC 0.013227605218655115 0.14194705583847272
BWI 0.013205945160437514 0.14171285000495995
IAD 0.013022968172385114 0.13975061821513868
CLE 0.012996524760204803 0.13946478256444914
JFK 0.012822879580061723 0.1376023185265247
PHL 0.012765686047556409 0.1369850797132589
TPA 0.012297439577818495 0.1319616169164865
FLL 0.0122760180564983

# Query 4

Page Rank implementation using raw GraphFrames

In [17]:
# Initalize all vertices with a score of 1 (1 / num_vertices is actually easier but the built in PageRank function does not do that)
v = flights_graph.vertices.withColumn("score", F.lit(1.0))
e = flights_graph.edges

In [18]:
iters = 10
damping = 0.85

for i in range(iters):
    # Calculate how much each node contributes to its neighbors and attach that score to the edges
    contributions = edges.join(v, edges.src == v.id) \
                    .select("dst", (F.col("score") / F.count("dst").over(Window.partitionBy("src"))).alias("contribution"))
    
    # Calculate the incoming contributions for each node
    incoming = contributions.groupBy("dst") \
        .agg(F.sum("contribution").alias("incoming"))
    
    # Apply damping as per the lecture and calculate the new scores
    dampened = incoming.withColumn("score", (1 - damping) + damping * F.col("incoming")) \
        .select(F.col("dst").alias("id"), "score")
    
    # Update the vertices (needs to be left_outer)
    # For vertices with no incoming edges, assign (1-damping) instead of 0 (to perfectly match the built-in PageRank implementation)
    v = vertices.join(dampened, "id", "left_outer") \
        .withColumn("score", F.coalesce(F.col("score"), F.lit(1 - damping))) \
        .select("id", "score")

In [19]:
# Get the final results from the custom implementation
custom_pagerank = v.select("id", F.col("score").alias("custom_score"))

built_in_result = flights_graph.pageRank(resetProbability=1-damping, maxIter=iters)
built_in_pagerank = built_in_result.vertices.select("id", F.col("pagerank").alias("built_in_score"))

# Join the results to compare
comparison = custom_pagerank.join(built_in_pagerank, "id")

comparison.orderBy("custom_score", ascending=False).show(10, False)

+---+------------------+------------------+
|id |custom_score      |built_in_score    |
+---+------------------+------------------+
|ATL|11.335268868250697|11.335268868250695|
|DFW|8.987200141119203 |8.987200141119203 |
|ORD|8.33083208401115  |8.33083208401115  |
|DTW|7.722346489398372 |7.722346489398374 |
|MSP|7.592985738022308 |7.59298573802231  |
|DEN|7.026287164606544 |7.026287164606544 |
|SLC|6.764494781487534 |6.764494781487534 |
|IAH|5.662321605756484 |5.6623216057564845|
|LAX|5.070949815830091 |5.070949815830092 |
|CVG|4.8567913760847174|4.856791376084718 |
+---+------------------+------------------+
only showing top 10 rows



## Results

The results show that the most "important" airport is the Hartsfield–Jackson Atlanta International Airport (ATL). This also matches with the fact that Atlanta is known to be the busiest airport in the US by quite a margin. All of the other airports in the top 10 are also considered to be very busy but the exact order differs slightly. For example, the Los Angeles International Airport (LAX) is considered to be the second busiest but it is ranked 9th by PageRank. This is likely because the edges in our graph are not weighted in any way (could use something like nr. of passengers per year).

# Query 5

Can we use built-in stuff for this?

The entire graph seems to be strongly connected (a directed path between all airports in both directions)

In [20]:
spark.sparkContext.setCheckpointDir("/tmp/spark-checkpoints")

strongly_connected = flights_graph.stronglyConnectedComponents(maxIter=10)
scc_sizes = strongly_connected.groupBy("component").count().orderBy("count", ascending=False)
scc_sizes.show(10, False)


+---------+-----+
|component|count|
+---------+-----+
|0        |296  |
+---------+-----+

